### Outline:
    Import Libraries and datasets from PyTorch Geometric
    1. Extracting subgraphs from set of nodes
    2. Extracting k_hop_subgraph from nodes
    3. Dropping a random walk with a probability
    4. MixHop

# Import Libraries

In [ ]:
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Numpy for matrices
import numpy as np
np.random.seed(0)

# 1. Extracting subgraphs from set of nodes

In [63]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import subgraph

# Load Cora dataset
dataset = Planetoid(root='.', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]
edge_index  = data.edge_index

# Example: Extract subgraph for nodes 0, 1, 2, 3
sampled_nodes = torch.tensor([0, 1, 2, 3])
subgraph_data = subgraph(sampled_nodes, edge_index, edge_attr=None)

print("Original Graph:")
print(data)

print("\nSubgraph:")
print(subgraph_data)

Original Graph:
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

Subgraph:
(tensor([[2, 1],
        [1, 2]]), None)


# 2. Extracting k_hop_subgraph from nodes

In [64]:
from typing import List, Optional, Tuple, Union
from torch import Tensor
from torch_geometric.utils.num_nodes import maybe_num_nodes

# Function to extract the k-hop subgraph around a given node or set of nodes
def k_hop_subgraph(
    node_idx: Union[int, List[int], Tensor],  # The target node(s)
    num_hops: int,  # The number of hops k
    edge_index: Tensor,  # The edge indices
    relabel_nodes: bool = False,  # Whether to relabel nodes to a contiguous range
    num_nodes: Optional[int] = None,  # The number of nodes in the graph
    flow: str = 'source_to_target',  # The flow direction ('source_to_target' or 'target_to_source')
    directed: bool = False,  # Whether the graph is directed
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:  # Returns the subgraph, edge indices, inverse mapping, and edge mask

    # Determine the number of nodes if not provided
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    # Ensure the flow direction is valid
    assert flow in ['source_to_target', 'target_to_source']
    if flow == 'target_to_source':
        row, col = edge_index
    else:
        col, row = edge_index

    # Initialize masks for nodes and edges
    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    # Convert node_idx to a tensor if it is not already
    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    # List to store the subsets of nodes at each hop
    subsets = [node_idx]

    # Perform k-hop expansion
    for _ in range(num_hops):
        node_mask.fill_(False)
        node_mask[subsets[-1]] = True
        torch.index_select(node_mask, 0, row, out=edge_mask)
        subsets.append(col[edge_mask])

    # Concatenate all subsets and get unique nodes
    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    # Create a mask for the subset of nodes
    node_mask.fill_(False)
    node_mask[subset] = True

    # If the graph is undirected, update the edge mask
    if not directed:
        edge_mask = node_mask[row] & node_mask[col]

    # Filter the edge index to include only the edges in the subgraph
    edge_index = edge_index[:, edge_mask]

    # Relabel nodes to a contiguous range if specified
    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]

    # Return the subset of nodes, the filtered edge index, the inverse mapping, and the edge mask
    return subset, edge_index, inv, edge_mask

# 3. Get subgraphs from target node 6 with 2 hops

In [65]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import subgraph

# Load Cora dataset
dataset = Planetoid(root='.', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]
edge_index  = data.edge_index

subset, edge_index, mapping, edge_mask = k_hop_subgraph(6, 2, edge_index, relabel_nodes=True)

In [66]:
print(edge_index)

tensor([[ 9, 17, 23, 27, 23, 23, 17, 19, 23, 26, 36, 27, 28, 37, 41, 27, 23, 23,
         27, 29,  0, 16, 17, 17, 23, 17, 23, 17, 40, 23, 30,  9, 17,  0,  3,  9,
         10, 12, 14, 16, 18, 19, 20, 21, 25, 26, 28, 33, 34, 35, 36, 37, 38, 40,
         42, 43, 17, 43,  3, 17, 23, 17, 17, 27,  0,  1,  2,  3,  6,  7, 11, 13,
         15, 19, 24, 27, 29, 30, 31, 32, 33, 34, 23, 17,  3, 17,  0,  4,  5,  8,
         22, 23, 37, 39, 40, 41,  4, 17, 37, 38,  8, 23, 15, 23, 23, 23, 17, 23,
         17, 23, 35, 36, 17, 34,  3, 17, 34,  4, 17, 27, 28, 17, 28, 42, 27, 14,
         17, 27,  4, 27, 17, 38, 17, 18],
        [ 0,  0,  0,  0,  1,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,  5,  6,  7,
          8,  8,  9,  9,  9, 10, 11, 12, 13, 14, 14, 15, 15, 16, 16, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 18, 18, 19, 19, 19, 20, 21, 22, 23, 23, 23, 23, 23, 23, 23, 23,
         23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 25, 26, 26, 27

# 4. Drop a random walk path

Drops edges from the adjacency matrix based on random walks.

The source nodes to start random walks from are sampled from the edge index with probability p, following a Bernoulli distribution.

## 4.1. Construct GCN model

In [67]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN_dropout_path(torch.nn.Module):
  """Graph Convolutional Network"""
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.gcn1 = GCNConv(dim_in, dim_h)
    self.gcn2 = GCNConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.5, training=self.training)
    h = self.gcn1(h, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.gcn2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

## 4.2. Dropout_path

In [68]:
from torch_geometric.utils import cumsum, degree, sort_edge_index, subgraph
from torch_geometric import is_compiling
import torch_geometric.typing

def dropout_path(edge_index: Tensor, p: float = 0.2, walks_per_node: int = 1,
                 walk_length: int = 3, num_nodes: Optional[int] = None,
                 is_sorted: bool = False, training: bool = True) -> Tuple[Tensor, Tensor]:
    # Ensure probability is within range
    if not (0.0 <= p <= 1.0):
        raise ValueError(f'Sample probability must be between 0 and 1 (got {p})')
    
    # Return unchanged edge_index if not in training mode or p=0
    if not training or p == 0.0:
        return edge_index, torch.ones(edge_index.size(1), dtype=torch.bool, device=edge_index.device)
    
    # Ensure required torch-cluster support is available
    if not torch_geometric.typing.WITH_TORCH_CLUSTER or is_compiling():
        raise ImportError('`dropout_path` requires `torch-cluster`.')
    
    # Sort edges if necessary
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    edge_orders = None
    if not is_sorted:
        edge_orders = torch.arange(edge_index.size(1), device=edge_index.device)
        edge_index, edge_orders = sort_edge_index(edge_index, edge_orders, num_nodes=num_nodes)
    
    # Randomly mask edges
    row, col = edge_index
    sample_mask = torch.rand(row.size(0), device=edge_index.device) <= p
    start = row[sample_mask].repeat(walks_per_node)
    
    # Perform random walk to determine paths
    rowptr = cumsum(degree(row, num_nodes=num_nodes, dtype=torch.long))
    n_id, e_id = torch.ops.torch_cluster.random_walk(rowptr, col, start, walk_length, 1.0, 1.0)
    e_id = e_id[e_id != -1].view(-1)  # Filter out illegal edges
    
    # Adjust for sorted edges if applicable
    if edge_orders is not None:
        e_id = edge_orders[e_id]
    
    # Apply mask to edges and return
    edge_mask = torch.ones(edge_index.size(1), dtype=torch.bool, device=edge_index.device)
    edge_mask[e_id] = False
    return edge_index[:, edge_mask], edge_mask


## 4.3. Train

In [69]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train_dropout_path(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 5

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        edge_index1, _ = dropout_path(data.edge_index, p= 0.2,walks_per_node = 1, walk_length = 3)
        _, out = model(data.x, edge_index1 )
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        # Print metrics every 10 epochs
        if(epoch % 1 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')

    return model

## 4.4. Test

In [70]:
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [71]:
%%time
import torch

# Set the device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create GCN model
gcn_dropout_path = GCN_dropout_path(dataset.num_features, 16, dataset.num_classes).to(device)
print(gcn_dropout_path)

# Train
train_dropout_path(gcn_dropout_path, data.to(device))

# Test
acc = test(gcn_dropout_path, data.to(device))
print(f'\nGCN test accuracy: {acc*100:.2f}%\n')

GCN_dropout_path(
  (gcn1): GCNConv(1433, 16)
  (gcn2): GCNConv(16, 7)
)
Epoch   0 | Train Loss: 1.945 | Train Acc:  18.57% | Val Loss: 1.95 | Val Acc: 14.00%
Epoch   1 | Train Loss: 1.941 | Train Acc:  25.71% | Val Loss: 1.94 | Val Acc: 17.40%
Epoch   2 | Train Loss: 1.934 | Train Acc:  27.14% | Val Loss: 1.94 | Val Acc: 19.40%
Epoch   3 | Train Loss: 1.927 | Train Acc:  35.00% | Val Loss: 1.94 | Val Acc: 21.80%
Epoch   4 | Train Loss: 1.918 | Train Acc:  37.14% | Val Loss: 1.93 | Val Acc: 25.80%
Epoch   5 | Train Loss: 1.908 | Train Acc:  42.86% | Val Loss: 1.93 | Val Acc: 29.00%

GCN test accuracy: 60.40%

CPU times: user 185 ms, sys: 38.4 ms, total: 223 ms
Wall time: 32.3 ms


# 5. MixHop Model

## 5.1. MixHopConv

In [72]:
import torch
from torch import nn, Tensor
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
from torch_geometric.utils import spmm
from typing import List, Optional

class MixHopConv(MessagePassing):
    def __init__(
        self,
        in_channels: int,  # Number of input features
        out_channels: int,  # Number of output features
        powers: Optional[List[int]] = None,  # List of powers for MixHop
        add_self_loops: bool = True,  # Whether to add self-loops
        bias: bool = True,  # Whether to add a bias term
        **kwargs,
    ):
        super().__init__(aggr='add', **kwargs)  # Initialize the MessagePassing class with 'add' aggregation
        
        self.powers = powers or [0, 1, 2]  # Default powers are [0, 1, 2]
        self.add_self_loops = add_self_loops  # Store the add_self_loops flag
        
        # Create a list of linear transformations for each power
        self.lins = nn.ModuleList([
            Linear(in_channels, out_channels, bias=False) if p in self.powers else nn.Identity()
            for p in range(max(self.powers) + 1)
        ])
        
        # Initialize the bias parameter if bias is True
        self.bias = Parameter(torch.empty(len(self.powers) * out_channels)) if bias else None
        self.reset_parameters()  # Reset parameters

    def reset_parameters(self):
        # Reset parameters of each linear transformation
        for lin in self.lins:
            if hasattr(lin, 'reset_parameters'):
                lin.reset_parameters()
        zeros(self.bias)  # Initialize the bias to zeros

    def forward(self, x: Tensor, edge_index, edge_weight=None) -> Tensor:
        # Normalize the edge index and edge weight using GCN normalization
        edge_index, edge_weight = gcn_norm(
            edge_index, edge_weight, x.size(0), False, self.add_self_loops, self.flow, x.dtype
        )
        
        # Initialize the output list with the transformed input features
        outs = [self.lins[0](x)]
        
        # Propagate the features through the graph for each power
        for lin in self.lins[1:]:
            x = self.propagate(edge_index, x=x, edge_weight=edge_weight)
            outs.append(lin(x))

        # Concatenate the outputs for each power along the feature dimension
        out = torch.cat([outs[p] for p in self.powers], dim=-1)
        
        # Add the bias term if it exists
        return out + self.bias if self.bias is not None else out

    def message(self, x_j: Tensor, edge_weight=None) -> Tensor:
        # Compute the message to be passed to the target nodes
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

    def message_and_aggregate(self, adj_t, x: Tensor) -> Tensor:
        # Perform sparse matrix multiplication to aggregate messages
        return spmm(adj_t, x, reduce=self.aggr)

    def __repr__(self):
        # Return a string representation of the MixHopConv layer
        return f'{self.__class__.__name__}({self.in_channels}, {self.out_channels}, powers={self.powers})'

## 5.2. Mixhop

In [73]:
from torch_geometric.nn import BatchNorm, Linear

class MixHop(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # First MixHopConv layer with powers [0, 1, 2] and 60 output features
        self.conv1 = MixHopConv(dataset.num_features, 60, powers=[0, 1, 2])
        # Batch normalization for the first layer's output
        self.norm1 = BatchNorm(3 * 60)

        # Second MixHopConv layer with powers [0, 1, 2] and 60 output features
        self.conv2 = MixHopConv(3 * 60, 60, powers=[0, 1, 2])
        # Batch normalization for the second layer's output
        self.norm2 = BatchNorm(3 * 60)

        # Third MixHopConv layer with powers [0, 1, 2] and 60 output features
        self.conv3 = MixHopConv(3 * 60, 60, powers=[0, 1, 2])
        # Batch normalization for the third layer's output
        self.norm3 = BatchNorm(3 * 60)

        # Linear layer to map the final output to the number of classes
        self.lin = Linear(3 * 60, dataset.num_classes)

    def forward(self, x, edge_index):
        # Apply dropout to the input features
        x = F.dropout(x, p=0.7, training=self.training)

        # First MixHopConv layer
        x = self.conv1(x, edge_index)
        # Apply batch normalization
        x = self.norm1(x)
        # Apply dropout
        x = F.dropout(x, p=0.9, training=self.training)

        # Second MixHopConv layer
        x = self.conv2(x, edge_index)
        # Apply batch normalization
        x = self.norm2(x)
        # Apply dropout
        x = F.dropout(x, p=0.9, training=self.training)

        # Third MixHopConv layer
        x = self.conv3(x, edge_index)
        # Apply batch normalization
        x = self.norm3(x)
        # Apply dropout
        x = F.dropout(x, p=0.9, training=self.training)

        # Final linear layer to get the class scores
        return self.lin(x)

## 5.3. Construct model

In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, data = MixHop().to(device), data.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5, weight_decay=0.005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.01)


## 5.4. Train function

In [75]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    scheduler.step()
    return float(loss)

## 5.5. Test function

In [76]:
@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs


In [77]:
best_val_acc = test_acc = 0
for epoch in range(1, 50):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Loss: 3.0828, Train: 0.1500, Val: 0.1600, Test: 0.1460
Epoch: 002, Loss: 4.4179, Train: 0.1714, Val: 0.2100, Test: 0.1860
Epoch: 003, Loss: 4.1751, Train: 0.1643, Val: 0.2100, Test: 0.1860
Epoch: 004, Loss: 4.9424, Train: 0.2714, Val: 0.2160, Test: 0.2020
Epoch: 005, Loss: 3.8162, Train: 0.1429, Val: 0.2160, Test: 0.2020
Epoch: 006, Loss: 6.1398, Train: 0.2143, Val: 0.2160, Test: 0.2020
Epoch: 007, Loss: 5.1311, Train: 0.1571, Val: 0.2160, Test: 0.2020
Epoch: 008, Loss: 5.9651, Train: 0.1500, Val: 0.2160, Test: 0.2020
Epoch: 009, Loss: 5.3913, Train: 0.1500, Val: 0.2160, Test: 0.2020
Epoch: 010, Loss: 5.8275, Train: 0.1500, Val: 0.2160, Test: 0.2020
Epoch: 011, Loss: 5.2305, Train: 0.2786, Val: 0.2160, Test: 0.2020
Epoch: 012, Loss: 4.4176, Train: 0.1500, Val: 0.2160, Test: 0.2020
Epoch: 013, Loss: 6.4840, Train: 0.2000, Val: 0.2160, Test: 0.2020
Epoch: 014, Loss: 4.3410, Train: 0.1714, Val: 0.2160, Test: 0.2020
Epoch: 015, Loss: 5.0022, Train: 0.3000, Val: 0.2160, Test: 0.